# TP MLFLOW
Florent Jakubowski

## A. Prise en main

1.Installez le package mlflow avec python dans un environnement virtuel.

In [1]:
#pip install mlflow

2.Dans un terminal lancez un serveur mlflow. Aller voir dans votre navigateur, sur le port correspondant, l'ui de mlflow.

In [2]:
#mlflow ui

Lorsque vous lancez mflow sans option par défaut mlflow va stocker toute la donnée dont il a besoin sur votre file system. Vous aurez notamment un dossier mlruns qui se créra par défaut.

3.Dans un notebook, utilisez le package mlflow pour vous connecter au serveur mlflow que vous avez lancé. Utilisez la bonne fonction pour paramétrer l'adresse du serveur. 

In [3]:
import mlflow

# Set the MLflow tracking URI to the address of your server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Example: Track a metric and parameter
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 10.2)

# To stop tracking in this run
mlflow.end_run()

c:\Ecole d'inge\Application of big data\venv\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Ecole d'inge\Application of big data\venv\lib\site-packages\pydantic\_internal\_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Attention ! A chaque fois que vous effectuerez une opération sur mlflow dans une autre cellule de votre notebook vous devrez vérifier avant que vous pointez bien sur le bon serveur mlflow. Il existe aussi une fonction pour connaître quelle adresse de serveur a été enregistrée. 

In [4]:
import mlflow

# Obtenir l'adresse actuelle du serveur MLflow
current_tracking_uri = mlflow.get_tracking_uri()
print(f"Adresse actuelle du serveur MLflow : {current_tracking_uri}")

# On peut changer l'adresse du serveur avec cette commande
# mlflow.set_tracking_uri("http://127.0.0.1:5000")

Adresse actuelle du serveur MLflow : http://127.0.0.1:5000


Créez une experiment via votre notebook ou avec via l'ui. 
Une experiment, ou une expérience en français, est un ensemble de run que vous avez effectué. Le but est de trouver in fine les meilleurs paramètres, modèles ou hyperparamètres pour votre besoin.

In [5]:
# Set the MLflow tracking URI to the address of your server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

id = mlflow.create_experiment("test")

RestException: RESOURCE_ALREADY_EXISTS: Experiment 'test' already exists.

Pareillement à l'adresse du serveur mlflow à chaque fois que vous exécuterez un run dans une cellule vous devrez définir l'experiment sur laquelle vous voulez envoyer votre run. Cherchez dans la documentation la fonction permettant de faire cela.

In [6]:
# Example: Track a metric and parameter
with mlflow.start_run(experiment_id=id):
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 10.2)
    

# To stop tracking in this run
mlflow.end_run()

MlflowException: Invalid experiment id: <built-in function id> of type <class 'builtin_function_or_method'>. Must be one of str, int, or None.

In [7]:
test = mlflow.get_experiment_by_name("tet")
type(test)
test==None

True

Nous vous proposons de créer une fonction `configure_experiment` permettant de créer une expérience ou de définir l'expérience si elle existe déjà.

In [8]:
def configure_experiment(name:str, params:dict={"",""},metrics:dict={"",""},tags:dict={"",""}, artifacts:list=[""]):

    # track du serveur
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    if mlflow.get_experiment_by_name(name) is None:
        
        return mlflow.create_experiment(name)
        
    else :
        # Récupérer l'ID de l'expérience
        return mlflow.get_experiment_by_name(name).experiment_id
"""
    # Commencer un nouveau run dans l'expérience existante
    with mlflow.start_run(experiment_id=experiment_id):
        for key,value in params:
             mlflow.log_param(key, value)
        for key,value in metrics:
            mlflow.log_metric(key, value)
        for key,value in tags:
            mlflow.set_tag(key, value)
        for path in artifacts:
            mlflow.log_artifact(path)
        # To stop tracking in this run
    mlflow.end_run()
"""
        
    

"\n    # Commencer un nouveau run dans l'expérience existante\n    with mlflow.start_run(experiment_id=experiment_id):\n        for key,value in params:\n             mlflow.log_param(key, value)\n        for key,value in metrics:\n            mlflow.log_metric(key, value)\n        for key,value in tags:\n            mlflow.set_tag(key, value)\n        for path in artifacts:\n            mlflow.log_artifact(path)\n        # To stop tracking in this run\n    mlflow.end_run()\n"

4.Après avoir créé l'experiment. Nous allons entraîner notre modèle. Prenez le dataset wine de la librairie sklearn et utilisez un algorithme de la famille des arbres de décisions. Nous allons lors de l'entraînement de notre modèle logger les mesures.   

Pour cela nous voulons lancer un nouveau run dans notre experiment sur mlflow. Le run correspond à un entraînement du modèle.   

-Créer une variable run_name pour le nouveau run que vous voulez créer avec la date du jour, l'heure, la minute et la seconde dans le nom.   
-Trouvez comment logger les paramètres d'entraînement, les hyperparamètres et les performances du modèle (metrics) explicitement dans mlflow en lançant votre run de manière manuelle.   
-Ajoutez également votre modèle avec un nom distinctif grâce à la méthode adéquat, vous devrez réutiliser ce nom lors des prochains entraînements. Que voyez-vous dans l'ui de mlflow ?   


Vous pouvez utiliser `with` pour ne pas avoir besoin d'utiliser la fonction `end_run()`

In [9]:
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import datetime

In [10]:
# Charger le dataset Wine
wine = datasets.load_wine()
X = wine.data
y = wine.target

In [11]:
# Diviser le dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paramètres de l'algorithme de l'arbre de décision
tree_params = {
    'criterion': 'gini',
    'max_depth': 3,
    'random_state': 42
}

# track du serveur
mlflow.set_tracking_uri("http://127.0.0.1:5000")
        
experiment_id = configure_experiment("wine_classification_experiment")

In [12]:
experiment_id

'183478325808655477'

In [13]:
with mlflow.start_run(experiment_id=experiment_id,run_name=f"run_{datetime.datetime.now().strftime('%Y/%m/%d_%H:%M:%S')}"):
    for key, value in tree_params.items():
        mlflow.log_param(key, value)
        
    # Initialiser et entraîner le modèle
    model = DecisionTreeClassifier(**tree_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
     # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques
    mlflow.log_metric("accuracy", accuracy)

    # Log des artefacts (le modèle)
    mlflow.sklearn.log_model(model, "model")

    # Ajouter le modèle avec un nom distinctif
    model_name = "decision_tree_model"
    mlflow.sklearn.log_model(model, model_name)



On peut utiliser `mlflow.active_run()` pour être sûr que le run est bien terminé.

In [14]:
# Afficher le résultat du run (ID du run)
print(mlflow.active_run())

None


Ecrivez un run sans utiliser un with et sans utilisez mlflow.end_run()

In [15]:
mlflow.start_run(experiment_id=experiment_id,run_name=f"run_{datetime.datetime.now().strftime('%Y/%m/%d_%H:%M:%S')}")
for key, value in tree_params.items():
        mlflow.log_param(key, value)
        
    # Initialiser et entraîner le modèle
model = DecisionTreeClassifier(**tree_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
    # Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
    # Log des métriques
mlflow.log_metric("accuracy", accuracy)

    # Log des artefacts (le modèle)
mlflow.sklearn.log_model(model, "model")

    # Ajouter le modèle avec un nom distinctif
model_name = "decision_tree_model"
mlflow.sklearn.log_model(model, model_name)

# Afficher le résultat du run (ID du run)
print(f"Run ID: {mlflow.active_run()}")


Run ID: <ActiveRun: >


La fonction `active_run()` nous retourne normalement ce run ci.

In [16]:
run = mlflow.active_run()
print(run)

<ActiveRun: >


Faisons bien attention de bien le fermer pour ne pas avoir de comportements exotiques ensuite.

Utilisez mlflow.end_run() et vérifiez avec active_run() qu'aucun run n'est retourné.

In [17]:
mlflow.end_run()
print(mlflow.active_run())

None


Il existe une autre manière de logger automatiquement des paramètres et des metrics à vous de la trouver.   
(Attention une fois activée cette fonction entrainera toujours un log automatique, veillez à la désactiver pour la suite)

In [18]:
from mlflow.sklearn import autolog
# Activer le suivi automatique pour scikit-learn
autolog()

In [19]:
mlflow.sklearn.autolog(disable=True)

5.Changez un ou plusieurs hyperparamètres et relancez un entraînement avec le même nom de modèle. Rendez-vous dans l'onglet model de mlflow que constatez-vous ? 


In [20]:
import mlflow
from mlflow.sklearn import autolog
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Charger le dataset Wine
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Diviser le dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Paramètres de l'algorithme de l'arbre de décision
tree_params = {
    'criterion': 'entropy',  # Changer un hyperparamètre
    'max_depth': 5,           # Changer un autre hyperparamètre
    'random_state': 42
}

# Configuration de l'expérience dans MLflow
experiment_name = "wine_classification_experiment"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = configure_experiment(experiment_name)

# Commencer un nouveau run avec le même nom de modèle
with mlflow.start_run(experiment_id=experiment_id,run_name=f"run_{datetime.datetime.now().strftime('%Y/%m/%d_%H:%M:%S')}"):
    for key, value in tree_params.items():
        mlflow.log_param(key, value)
        
    # Initialiser et entraîner le modèle
    model = DecisionTreeClassifier(**tree_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
     # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques
    mlflow.log_metric("accuracy", accuracy)

    # Log des artefacts (le modèle)
    mlflow.sklearn.log_model(model, "model")

    # Ajouter le modèle avec un nom distinctif
    model_name = "decision_tree_model"
    mlflow.sklearn.log_model(model, model_name)


Vous pouvez récupérer le dernier run avec la fonction last_active_run()

In [21]:
run = mlflow.last_active_run() 
print(run)

<Run: data=<RunData: metrics={'accuracy': 0.9166666666666666}, params={'criterion': 'entropy', 'max_depth': '5', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "3ca200ab28e74974b6d57f69a393ffbd", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-11-15 13:10:06.106951", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.18", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '

6.Entraîner 3,4 modèles avec des hyperparamètres différents.


In [22]:
import mlflow
from mlflow.sklearn import autolog
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Charger le dataset Wine
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Diviser le dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Paramètres de l'algorithme de l'arbre de décision
tree_params = {
    'criterion': 'log_loss',  # Changer un hyperparamètre
    'max_depth': 9,           # Changer un autre hyperparamètre
    'random_state': 36
}

# Configuration de l'expérience dans MLflow
experiment_name = "wine_classification_experiment"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = configure_experiment(experiment_name)

# Commencer un nouveau run avec le même nom de modèle
with mlflow.start_run(experiment_id=experiment_id,run_name=f"run_{datetime.datetime.now().strftime('%Y/%m/%d_%H:%M:%S')}"):
    for key, value in tree_params.items():
        mlflow.log_param(key, value)
        
    # Initialiser et entraîner le modèle
    model = DecisionTreeClassifier(**tree_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
     # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques
    mlflow.log_metric("accuracy", accuracy)

    # Log des artefacts (le modèle)
    mlflow.sklearn.log_model(model, "model")

    # Ajouter le modèle avec un nom distinctif
    model_name = "decision_tree_model"
    mlflow.sklearn.log_model(model, model_name)


In [23]:
import mlflow
from mlflow.sklearn import autolog
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Charger le dataset Wine
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Diviser le dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Paramètres de l'algorithme de l'arbre de décision
tree_params = {
    'criterion': 'gini',  # Changer un hyperparamètre
    'max_depth': 13,           # Changer un autre hyperparamètre
    'random_state': 50
}

# Configuration de l'expérience dans MLflow
experiment_name = "wine_classification_experiment"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment_id = configure_experiment(experiment_name)

# Commencer un nouveau run avec le même nom de modèle
with mlflow.start_run(experiment_id=experiment_id,run_name=f"run_{datetime.datetime.now().strftime('%Y/%m/%d_%H:%M:%S')}"):
    for key, value in tree_params.items():
        mlflow.log_param(key, value)
        
    # Initialiser et entraîner le modèle
    model = DecisionTreeClassifier(**tree_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
     # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques
    mlflow.log_metric("accuracy", accuracy)

    # Log des artefacts (le modèle)
    mlflow.sklearn.log_model(model, "model")

    # Ajouter le modèle avec un nom distinctif
    model_name = "decision_tree_model"
    mlflow.sklearn.log_model(model, model_name)


## B. Versionning des données avec DVC

Pour chaque run le dataset utilisé est précisé dans mlflow. Mais les informations fournies sont assez pauvres. Pour améliorer ça nous allons utiliser l'outil dvc en combinaison avec mlflow.

dvc fonctionne de pair avec git.

Initialisez un repo git localement.

In [24]:
!git init

Reinitialized existing Git repository in C:/Ecole d'inge/Application of big data/.git/


Installez dvc avec pip

In [25]:
!pip install dvc


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Initialisez un projet dvc.

In [26]:
!dvc init

ERROR: failed to initiate DVC - c:\Ecole d'inge\Application of big data\.dvc is ignored by your SCM tool. 
Make sure that it's tracked, for example, by adding '!.dvc' to .gitignore.


Regardez ce qui a été créé avec un git status.

In [27]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvcignore
	modified:   .gitignore
	modified:   Mlflow_tp_efrei_etudiant.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Ajoutez les nouveaux fichiers dans le fichier .dvc : le .gitignore, le fichier config, et le .dvcignore.

In [28]:
!git commit -m "initialize repo"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvcignore
	modified:   .gitignore
	modified:   Mlflow_tp_efrei_etudiant.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Nous pouvons ajouter un stockage distant dans le cloud. Ici nous allons utiliser un stockage dans un dossier local pour les besoins du tp.

In [29]:
!dvc remote add -d dvc-remote /tmp/dvc-storage

Setting 'dvc-remote' as a default remote.


ERROR: configuration error - config file error: remote 'dvc-remote' already exists. Use `-f|--force` to overwrite it.


On peut regarder le contenu du fichier dvc config.

In [30]:
#cat .dvc/config
Get-Content .dvc/config

NameError: name 'Get' is not defined

Nous constatons que l'url locale a bien été ajouté. Nous pouvons commiter ces changements à git.

In [31]:
!git commit .dvc/config -m "add remote storage"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .dvcignore
	modified:   .gitignore
	modified:   Mlflow_tp_efrei_etudiant.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Nous allons créer un dossier data pour stocker notre dataset. 

In [32]:
!mkdir data

Un sous-r�pertoire ou un fichier data existe d�j�.


Télécharger notre dataset et le placer dans le dossier data.

In [33]:
import requests
import zipfile
import io
import pandas as pd


dataset_source_url = "https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip"

content = requests.get(dataset_source_url).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

raw_data.to_csv(path_or_buf="data/hour.csv")

Affichons le contenu de data.

In [34]:
ls data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 2E08-72D9

 R�pertoire de c:\Ecole d'inge\Application of big data\data

15/11/2023  14:10    <DIR>          .
15/11/2023  13:57    <DIR>          ..
15/11/2023  13:21                11 .gitignore
15/11/2023  14:10         1�161�688 hour.csv
15/11/2023  13:28                90 hour.csv.dvc
               3 fichier(s)        1�161�789 octets
               2 R�p(s)  43�476�619�264 octets libres


Si nous voulons commencer à suivre les changements d'un fichier il nous suffit de l'ajouter via dvc.

In [35]:
!dvc add data/hour.csv

⠋ Checking graph




To track the changes with git, run:

	git add 'data\hour.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


Nous pouvons regarder de nouveau ce qu'il y a dans notre dossier.

In [36]:
ls -l data

 Le volume dans le lecteur C s'appelle OS

Fichier introuvable



 Le num�ro de s�rie du volume est 2E08-72D9

 R�pertoire de c:\Ecole d'inge\Application of big data


 R�pertoire de c:\Ecole d'inge\Application of big data\data

15/11/2023  14:10    <DIR>          .
15/11/2023  13:57    <DIR>          ..
15/11/2023  13:21                11 .gitignore
15/11/2023  14:10         1�161�688 hour.csv
15/11/2023  14:10                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  43�475�259�392 octets libres


Nous voyons un nouveau fichier .dvc.   
Si nous regardons à l'intérieur : 

In [38]:
!Get-Content data/hour.csv.dvc

'Get-Content' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


On voit que le fichier contient des informations à propos de notre csv :  
- Un hash du fichier 
- l'algorithme de hashage utilisé
- la taille
- le chemin

Un gitignore a été créé par défaut, si on regarde à l'intérieur : 

In [40]:
!Get-Content data/.gitignore

'Get-Content' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


On peut voir que notre csv y est renseigné.

Maintenant ajoutons le nouveau fichier data/hour.csv.dvc et le fichier data/.gitignore

In [41]:
!git add data/.gitignore data/hour.csv.dvc

et commitons le

In [42]:
!git commit -m "add .dvc file to track hours.csv file"

[master bf79dbb] add .dvc file to track hours.csv file
 1 file changed, 2 insertions(+), 2 deletions(-)


Une bonne idée est de créer un tag pour chaque version de notre dataset.

In [43]:
!git tag -a 'v1' -m 'raw_data'

fatal: tag ''v1'' already exists


Notre donnée est toujours sur notre dossier en local, maintenant nous devons l'envoyer sur notre stokage distant (qui pour rappel et en fait un autre dossier local). Pour ça nous utilisons la commande dvc push.

In [44]:
!dvc push

Everything is up to date.


On peut regarder dans notre "remote storage" ce que nous avons :

In [45]:
ls -lR /tmp/dvc-storage

Le format du param�tre est incorrect - "mp".


Nous pouvons voir que notre fichier est présent dans le dossier mais avec un nom différent, ce nom correspond au hash de la donnée du fichier.

Maintenant que nos données sont sauvegardées à distance, nous pouvons les supprimer localement. Sauf le fichier .dvc ! Car sinon vous perdrez le lien avec vos données.

In [46]:
ls data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 2E08-72D9

 R�pertoire de c:\Ecole d'inge\Application of big data\data

15/11/2023  14:10    <DIR>          .
15/11/2023  13:57    <DIR>          ..
15/11/2023  13:21                11 .gitignore
15/11/2023  14:10         1�161�688 hour.csv
15/11/2023  14:10                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  43�478�331�392 octets libres


In [48]:
!rm -r data/hour.csv

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [49]:
!ls data

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Un autre emplacement où vos données résident est le dossier .dvc/cache

In [50]:
!ls .dvc/cache/files/md5/0a/1c63297d478edfdcc18433bb509cd5
!Get-Content .dvc/cache/files/md5/0a/1c63297d478edfdcc18433bb509cd5

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'Get-Content' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Nous supprimons aussi les données à l'intérieur.

In [51]:
!rm -r .dvc/cache

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Si nous voulons récupérer nos données localement, nous pouvons utiliser dvc pull pour récupérer les données.

In [52]:
!dvc pull

Everything is up to date.


Si nous regardons dans le dossier data :

In [53]:
ls -l data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 2E08-72D9

 R�pertoire de c:\Ecole d'inge\Application of big data


 R�pertoire de c:\Ecole d'inge\Application of big data\data

15/11/2023  14:10    <DIR>          .
15/11/2023  13:57    <DIR>          ..
15/11/2023  13:21                11 .gitignore
15/11/2023  14:10         1�161�688 hour.csv
15/11/2023  14:10                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  43�477�733�376 octets libres


Fichier introuvable


Notre fichier est de retour.

Maintenant modifions nos données !

In [54]:
ls data/hour.csv

Option non valide - "hour.csv".


In [56]:
!(Get-Content -Path "data/hour.csv" -Raw) -replace "(?m)^.*\r?\n", "" | Set-Content -Path "data/hour.csv"

-replace �tait inattendu.


In [57]:
!ls -l data/hour.csv

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Et répétons les opérations précédantes : 

In [58]:
!dvc add data/hour.csv


To track the changes with git, run:

	git add 'data\hour.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



In [59]:
!git add data/hour.csv.dvc

In [60]:
!git commit -m 'data: remove 1000 lines'

error: pathspec 'remove' did not match any file(s) known to git
error: pathspec '1000' did not match any file(s) known to git
error: pathspec 'lines'' did not match any file(s) known to git


In [61]:
!git tag -a 'v2' -m 'removed 1000 lines'

fatal: too many arguments


In [62]:
!dvc push

Everything is up to date.


In [63]:
!rm -r data/hour.csv

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [64]:
!rm -rf .dvc/cache

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


Nous pouvons maintenant regarder dans notre git log, et voir l'historique des modifications.

In [65]:
!git log

commit bf79dbbafba76f1d812ed9849ef932fa0bf391ec
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 14:11:08 2023 +0100

    add .dvc file to track hours.csv file

commit 73490a1a4d8c2e34652265a53de3d2c276c4fcdd
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 14:07:49 2023 +0100

    test1

commit c851f3c9d8f7ab0160cfef03ca7e001663d5ac03
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 14:06:19 2023 +0100

    test

commit ee3cae28bceff5f10aceb7db628d045cb5cbe2b3
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 13:58:34 2023 +0100

    test

commit 2fc018930fdde1f12113fa77e531147b45eb551f
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 13:23:01 2023 +0100

    add .dvc file to track hours.csv file

commit e5664c16479717c59ea4484ad6e8d54d32a48ed4
Author: STANG94 <stephane.tang@efrei.net>
Date:   Wed Nov 15 13:21:01 2023 +0100

    add remote storage

commit 3b06766748f86bcc54dd6b0c9ade031020721bcf
Author: STANG94 <stephane.tang@

Pour accéder et extraire des versions spécifiques de nos données nous pouvons utiliser le package dvc en python. 

In [66]:
!pip freeze | Select-String -Pattern "dvc"

"""
Output
dvc==3.29.0
dvc-data==2.20.0
dvc-http==2.30.2
dvc-objects==1.2.0
dvc-render==0.6.0
dvc-studio-client==0.15.0
dvc-task==0.3.0
"""

'Select-String' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


'\nOutput\ndvc==3.29.0\ndvc-data==2.20.0\ndvc-http==2.30.2\ndvc-objects==1.2.0\ndvc-render==0.6.0\ndvc-studio-client==0.15.0\ndvc-task==0.3.0\n'

In [67]:
import dvc
import dvc.api

In [68]:
path="data/hour.csv.dvc"
repo="dvc-remote"
version="bf79dbbafba76f1d812ed9849ef932fa0bf391ec"

#revision can be git commit id, commit tag, ...

data_url = dvc.api.get_url(
    path=path, 
    repo=repo,
    rev=version
)

CloneError: SCM error

In [ ]:
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

In [ ]:
print(MLFLOW_TRACKING_URI)

http://127.0.0.1:5000


In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
configure_experiment(name="ML_EXP_WITH_DVC")

'389645318088192245'

In [ ]:
import pandas as pd

data = pd.read_csv(data_url, sep=",")
print(data.head())

       dteday  instant  season  yr  mnth  hr  holiday  weekday  workingday  \
0  2011-01-01        1       1   0     1   0        0        6           0   
1  2011-01-01        2       1   0     1   1        0        6           0   
2  2011-01-01        3       1   0     1   2        0        6           0   
3  2011-01-01        4       1   0     1   3        0        6           0   
4  2011-01-01        5       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


In [ ]:
from datetime import datetime, time

data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [ ]:
data.head()

,dteday,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Nous allons maintenant utiliser 2 mécanismes pour ajouter plus d'informations sur notre jeu de données dans MLFlow :    
-Grâce à dvc, nous avons maintenant des liens vers les différentes versions de notre jeu de données.   
-Nous pouvons l'utiliser en combinaison avec le module mlflow.data pour ajouter plus d'informations sur notre jeu de données.   

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
start_date = '2011-01-01 00:00:00'
end_date = '2011-01-28 23:00:00'
dataset = data.loc[start_date:end_date]

In [ ]:
print(dataset)

                         dteday  instant  season  yr  mnth  hr  holiday  \
2011-01-01 00:00:00  2011-01-01        1       1   0     1   0        0   
2011-01-01 01:00:00  2011-01-01        2       1   0     1   1        0   
2011-01-01 02:00:00  2011-01-01        3       1   0     1   2        0   
2011-01-01 03:00:00  2011-01-01        4       1   0     1   3        0   
2011-01-01 04:00:00  2011-01-01        5       1   0     1   4        0   
...                         ...      ...     ...  ..   ...  ..      ...   
2011-01-28 19:00:00  2011-01-28      614       1   0     1  19        0   
2011-01-28 20:00:00  2011-01-28      615       1   0     1  20        0   
2011-01-28 21:00:00  2011-01-28      616       1   0     1  21        0   
2011-01-28 22:00:00  2011-01-28      617       1   0     1  22        0   
2011-01-28 23:00:00  2011-01-28      618       1   0     1  23        0   

                     weekday  workingday  weathersit  temp   atemp   hum  \
2011-01-01 00:00:00    

Dans la même cellule : 

1)
- Aller voir la documentation du module mlflow data et importer le bon objet pour les données pandas.  
- Créer un run avec la date, l'heure, etc. comme nom.    
- pour l'entrainement vous pouvez utiliser le chemin vers votre dataset lors de la création de votre dataframe.   

2) 
- logger le dataset avec la méthode appropriée.  
- logger également le chemin vers votre dataset.   
- logger la version du dataset utilisée.   

3)
- Créer un fichier texte et logger le en tant qu'artifact. Dans ce fichier vous pourrez indiquer la colonne qui a servi de target, les features numériques et les features catégorielles.

4)
- N'oubliez pas d'utiliser la fonction mlflow.end_run() si vous n'avez pas utilisez de with pour le run.  

## C. Déploiement

7.Trouver comment transitionner un modèle en état staging et ensuite dans l'état production. 

8.Créer une fonction qui récupère la version du modèle avec les meilleurs metrics d'entraînement et qui transitionne ce modèle dans l'état production.

9. Le serveur mlflow peut vous fournir des prédictions à partir des modèles enregistrés. Faites en sorte d'obtenir une prédiction de votre dernier modèle en requêtant le serveur mlflow. (Voir : https://mlflow.org/docs/latest/models.html#command-line-interface)

10.Créer un script à part qui pull le dernier modèle depuis le model registry. Plus tard vous pourrez utiliser ce script pour récupérer le modèle dans une api.

## D. Entraînement d'un CNN et log des metrics dans MLFlow

Voir le notebook donnée par le formateur.